In [1]:
import numpy as np
import seaborn as sns
import random
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = "retina"

In [2]:
from importlib import reload

In [3]:

########################################

import math
from typing import NamedTuple, List, Tuple, Dict, Any

import pysam
import numpy as np

rng = np.random.default_rng()
# import scipy.stats

from numpy.typing import NDArray
# import pandas as pd

import tqdm
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams["svg.fonttype"] = 'none'

# import array
import gzip
import io
import sys
from argparse import ArgumentParser
import re
import random
import json
import itertools

# import pdb


#######

from src.config import *
from src.utils import *

from src.coverage import MyFastaFile, MyAlignmentFile, Quality
# from src.figures import *
from src.quality import *
from src.pangene import *
# from src.sampling import *
from src.summary import *
from src.updateBinning import *

from src.mt import *
from src.lambdaDNA import *
from src.plastid import *

#############################
#### config parameters
#############################


# data exported to Jinja
# data = {}
# config_params()
def config_params_further():
        
    SAMPLE = re.search(r'/?([^/\.]+)\.?[^/]*$', params['bamfile']).groups()[0]
    # print(SAMPLE)
    data['SAMPLE'] = SAMPLE

    # nuclear bin size 
    a, b = prefixBpSize(params['binSize'])
    if a - np.ceil(a) < 1e-6:
        data['bin_size'] = f'{int(a)} {b}'
    else:
        data['bin_size'] = f'{a} {b}'
    data['bin_size']

    fa = MyFastaFile(params['fafile'])
    params['fa'] = fa

    reference_length = fa.get_reference_length
    params['reference_length'] = reference_length

    # chrs to sample and compt
    chrs_valid = params['testchrs'].split(',')
    # lengths of chrs
    lens_valid = [reference_length(chr) for chr in chrs_valid]
    params['chrs_valid'] = chrs_valid
    params['lens_valid'] = lens_valid

    chr_lambda = params['chr_lambda']
    bins_lambda = params['bins_lambda']
    chr_MT = params['chr_MT']

    binSize_lambda = getBins(reference_length(chr_lambda), bins_lambda)
    binSize_MT = getBins(reference_length(chr_MT), bins_lambda)
    binSizeContig = {chr_lambda: binSize_lambda[0], chr_MT: binSize_MT[0]}
    binsContig = {chr_lambda: binSize_lambda[1], chr_MT: binSize_MT[1]}
    chrs_alias = {'lambda': chr_lambda, 'MT': chr_MT}

    params['binSize_lambda'] = binSize_lambda
    params['binSize_MT'] = binSize_MT
    params['binSizeContig'] = binSizeContig
    params['binsContig'] = binsContig
    params['chrs_alias'] = chrs_alias
    
    bam = MyAlignmentFile(params['bamfile'], 'rb')
    params['bam'] = bam

    # classes
    # ('chr1', 23) -> BinCov()
    dict_binning = dict()
    params['dict_binning'] = dict_binning

    # 'XXCGXX' -> KmerCov()
    dict_cgkmer = dict()
    params['dict_cgkmer'] = dict_cgkmer

    # quality stats
    quality = Quality()
    params['quality'] = quality

    params['reads_to_sample'] = min([params['reads_to_sample'], bam.mapped])
    

#######################################
#### summarization and plots
#######################################

config_params_further()

# DNA content
compt_plot_DNA_content()

# # base and read quality
# compt_quality()
# plot_read_length()
# plot_base_quality()
# plot_read_map_quality()
# plot_bar_base_cigar()

# # pangene
# pangene_sampling()
# pangene_compt_plot_meth()

# # MT
# compt_MT()
# if data['mt_is_covered'] == 1:
#     plot_mt_depth_binning()
#     plot_mt_base_error_rate()

# # lambda
# compt_lambda()
# if data['lambda_is_covered'] == 1:
#     plot_lambda_depth_binning()
#     plot_lambda_base_error_rate()

# # plastid
# compt_plastid()
# if data['plastid_is_covered'] == 1:
#     plot_plastid_depth_binning()
#     plot_plastid_base_error_rate()


# print(params)
print(data)


{'SAMPLE': 'ENCFF873NOV', 'bin_size': '100 Kbp', 'lambda_DNA_content_vs_nuclear': '0.2145', 'lambda_DNA_content_vs_MT': '38'}


In [6]:
import src.summary
reload(src.summary)

<module 'src.summary' from '/root/bsDoctor/src/summary.py'>

In [7]:
src.summary.compt_plot_DNA_content()

In [8]:
import pickle

In [11]:
with open("./data.pickle", 'rb') as fd:
    data = pickle.load(fd)

In [24]:
import bsDoctor
reload(bsDoctor)

<module 'bsDoctor' from '/root/bsDoctor/bsDoctor.py'>

In [25]:
bsDoctor.write_report()

In [ ]:
from argparse import

In [ ]:
class MyArgumentParser(ArgumentParser):
    def __init__(self, **args) -> None:
        super().__init__(**args)
